# A2DI - TP4

## Exo 1 - Comparaison de classifieurs

(Question 1)

In [1]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np

# Permet l'affichage des graphiques

%matplotlib inline

# Variable utiles

nb_features = 64

data=datasets.load_digits()
X=data.data
X=X.T
c=data.target

def show_img56(im, l):
    img = np.array(im).reshape((8, 8))
    img = np.array(list(map(lambda x : list(map(lambda y : [y, y, y], x)), img)))
    plt.title("6" if (l == 1.) else "5")
    plt.imshow(img, cmap=plt.cm.gray_r, interpolation='nearest')

(Question 2)

In [2]:
def extract_56():
    X1 = []
    c1 = []
    for i in range(len(c)):
        if c[i] == 5 or c[i] == 6:
            X1.append(data.data[i])
            c1.append(data.target[i])
    return np.array(X1).T, np.array(c1)

X1, c1 = extract_56()

(Question 3)

In [3]:
# -1 == 5 & 1 == 6
c1 = np.sign(c1-np.mean(c1))

(Question 4)

In [4]:
def kfold_data(X, c, k):
    x = np.array(X).T

    # On trie les 1 et les -1
    arplus = []
    arless = []
    
    for i in range(len(x)):
        if c[i] == 1:
            arplus.append(x[i])
        else:
            arless.append(x[i])

    # On calcule la taille des plis
    size_plus = len(arplus)//k
    rest_plus = len(arplus)%k

    size_less = len(arless)//k
    rest_less = len(arless)%k
    
    # On crée tous les plis
    k_plis = []

    for i in range(k):
        x = arplus[i*size_plus:(i+1)*size_plus] + arless[i*size_less:(i+1)*size_less]
        y = [1 for _ in range(size_plus)] + [-1 for _ in range(size_less)]
        s = np.random.get_state()
        np.random.shuffle(x)
        np.random.set_state(s)
        np.random.shuffle(y)
        k_plis.append([x, y])

    # On ajoute le reste au dernier plis
    k_plis[k-1][0] += arplus[k*size_plus:] + arless[k*size_plus:]
    k_plis[k-1][1] += [1 for _ in range(rest_plus)] + [-1 for _ in range(rest_less)]

    # On crée enfin toutes les possibilités de répartition des plis
    # Taille (4, k, sizeX/k)
    plis = [[] for _ in range(4)]
    
    for i in range(len(k_plis)):
        x = []
        y = []
        for j in range(len(k_plis)):
            if j != i:
                x += k_plis[j][0]
                y += k_plis[j][1]
        plis[0].append(np.array(x).T)
        plis[1].append(np.array(y))
        plis[2].append(np.array(k_plis[i][0]).T)
        plis[3].append(np.array(k_plis[i][1]))

    return plis[0], plis[1], plis[2], plis[3]

k = 5
t = kfold_data(X1, c1, k)

(Question 5 et 6)

In [5]:
# Fonction ptest qui évalue un x selon un theta
def ptest(x, theta):

    # Ajout du 1 à chaque x
    npx = np.insert(np.array(x), nb_features, 1, axis=1)

    # On met theta comme il faut
    theta = theta.reshape((1, nb_features+1))
    
    # Calcul
    v = np.dot(theta, npx.T)

    # On applique la fonction signe
    v = np.sign(v)

    return v[0][0]

# Fonction ptrain
def ptrain_v2(X_train, c_train, iterations):   
    error_train = []
    theta = np.array([np.random.random() for _ in range(nb_features+1)])

    for _ in range(iterations):
        cpt = 0
        for i in range(len(c_train)):
            x = [X_train[j][i] for j in range(nb_features)]
            c = ptest([x], theta)

            # Erreur
            if (c == 1 and c_train[i] == -1) or (c == -1 and c_train[i] == 1):
                theta -= c * np.concatenate((np.array(x),np.array([1])))
                cpt += 1
        
        cpt = (cpt/len(c_train))*100
        error_train.append(cpt)

    #plt.plot(error_train, label='Erreur de train')
    #plt.yticks((0, 10))
    #plt.legend()
    #plt.show()
    
    return theta

#for i in range(5):
#    print("Pli " + str(i))
#    ptrain_v2(t[0][i], t[1][i], 10)

#print(ptest([[X1[j][21] for j in range(nb_features)]], theta))
#show_img56(X1.T[21], c1[21])

(Question 7)

In [6]:
def get_eval(plis, nb_pli):
    th = ptrain_v2(plis[0][nb_pli], plis[1][nb_pli], 10)
    
    error = 0
    m = [0 for _ in range(4)]
    cpt = 0
    for i in range(len(plis[3][nb_pli])):
            x = [plis[2][nb_pli][j][i] for j in range(nb_features)]
            c = ptest([x], th)
            cpt += 1

            # Vrai positif
            if c == 1 and plis[3][nb_pli][i] == 1:
                m[0] += 1
            # Faux positif
            elif c == 1 and plis[3][nb_pli][i] == -1:
                m[1] += 1
                error += 1
            # Vrai négatif
            elif c == -1 and plis[3][nb_pli][i] == -1:
                m[2] += 1
            # Faux négatif
            else:
                m[3] += 1
                error += 1
    m = list(map(lambda x : x / cpt, m))
                
    error = (error / len(plis[3][nb_pli])) * 100
    
    return error, m

(Question 8)

In [30]:
def loop_cross_validation():
    error_mean = 0
    m_mean = [0 for _ in range(4)]
    for i in range(k):
        er, m = get_eval(t, i)
        error_mean += er
        for j in range(4):
            m_mean[j] += m[j]
    
    m_mean = list(map(lambda x : x/k, m_mean))
    error_mean /= k
    
    return error_mean, m_mean
        
print(loop_cross_validation())

(0.8333333333333333, [0.49033333333333334, 0.0, 0.5013333333333334, 0.008333333333333333])
